<a href="https://colab.research.google.com/github/mehrn79/text_classification/blob/train_model/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hazm
!pip install stopwords_guilannlp
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 17.8 MB/s 
     |████████████████████████████████| 233 kB 47.8 MB/s 
     |████████████████████████████████| 1.4 MB 40.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=292a9b9f01e5bcfa6a1c1769ba7cb4bc26e4cb588de0e400f3a5bd45b2fff1fd
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=155239 sha256=62657d9f6384ee77ab68d8cd519450ac487f9997d675e5a79038e90b5e2c1396
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
Looking in indexes: https:/

In [3]:
import json
import nltk
import numpy as np
# from __future__ import unicode_literals
from hazm import *
import pandas as pd
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import collections
import re
from collections import Counter, defaultdict
from stopwords_guilannlp import stopwords_output
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Convolution1D
from keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras import optimizers
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.utils.vis_utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
import codecs
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/MyDrive/chart.csv' chart.csv

Mounted at /content/drive


In [5]:
with open('/content/chart.csv') as f:
  chart = f.read()

In [6]:
chart = pd.read_csv('chart.csv')
text = chart['clean_text'].values
Categories= chart['Cat'].values

In [7]:
category_le = LabelEncoder()
Categories = category_le.fit_transform(Categories)

category_x = text

category_y = Categories


In [8]:
x_cat_train, x_cat_test, y_cat_train, y_cat_test = train_test_split(category_x, category_y, test_size=0.20)
print('--Category Predictor--')
print('Train size: ',len(x_cat_train))
print('Test size : ',len(x_cat_test))

--Category Predictor--
Train size:  132180
Test size :  33045


In [9]:
cat_vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1), min_df=0.005)
# train set
x_cat_train_corpus = x_cat_train
# test set
x_cat_test_corpus = x_cat_test

# Fit vectorizer
cat_vectorizer.fit(x_cat_train_corpus)

# Print vocabulary
cat_vocab = cat_vectorizer.get_feature_names()
print(cat_vocab)
print('Vocab length:', len(cat_vocab))

# Transform vectorizer over train and test set
x_cat_train_vec = cat_vectorizer.transform(x_cat_train_corpus).toarray()
x_cat_test_vec = cat_vectorizer.transform(x_cat_test_corpus).toarray()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['آئين', 'آب', 'آباد', 'آبادي', 'آبهاي', 'آبي', 'آبياري', 'آت', 'آتي', 'آثار', 'آثاري', 'آحاد', 'آخرين', 'آد', 'آداب', 'آدمي', 'آذر', 'آذربايج', 'آذرماه', 'آر', 'آرا', 'آراء', 'آرام', 'آراي', 'آرزو', 'آرزوي', 'آرمانهاي', 'آرژانتين', 'آزاد', 'آزادسازي', 'آزادي', 'آزادگ', 'آزار', 'آزماي', 'آزمايشي', 'آزمايشگاه', 'آزمون', 'آس', 'آساني', 'آساي', 'آستانه', 'آسوشيتدپرس', 'آسيا', 'آسياي', 'آسيايي', 'آسيب', 'آسيبهاي', 'آسيبپذير', 'آشاميدني', 'آشتي', 'آشكار', 'آشنا', 'آشنايي', 'آغاز', 'آف', 'آفتاب', 'آفتابي', 'آفريقا', 'آفريقاي', 'آفريقايي', 'آفرين', 'آقا', 'آقاي', 'آل', 'آلاينده', 'آلماني', 'آلوده', 'آلودگي', 'آماده', 'آمادگي', 'آمار', 'آمارهاي', 'آماري', 'آمد', 'آمدن', 'آمدند', 'آمده', 'آمده_اس', 'آمده_اند', 'آمده_بود', 'آمريكا', 'آمريكاي', 'آمريكايي', 'آموز', 'آموزشي', 'آموزي', 'آميز', 'آميزي', 'آنجايي', 'آنكارا', 'آنكه', 'آنهايي', 'آهن', 'آهنگ', 'آواز', 'آور', 'آورد', 'آوردن', 'آوردند', 'آورده', 'آورده_اس', 'آورده_اند', 'آورند', 'آوري', 'آوريل', 'آي', 'آيد', 'آيند', 'آينده', 'آينه', 'آيين',

In [11]:
titles=[]
test_titles=[]

for data in x_cat_train :
  arr=data.split(' ')
  s=' '.join(arr[:25])
  titles.append(s)

for data in x_cat_test :
  arr=data.split(' ')
  s=' '.join(arr[:25])
  test_titles.append(s)

In [12]:
lstm_x_train = titles

# test set
lstm_x_test = test_titles


In [13]:
num_words = 10000

# create the tokenizer
tokenizer = Tokenizer(num_words=num_words)

# fit the tokenizer on the documents
tokenizer.fit_on_texts(x_cat_train_corpus)

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(lstm_x_train)

# pad sequences
max_length = max([len(s.split()) for s in lstm_x_train])
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

encoded_docs = tokenizer.texts_to_sequences(lstm_x_test)
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [14]:
y_cat_train_classes = np.unique(y_cat_train)
y_cat_train_classes_len = len(y_cat_train_classes)

y_cat_test_classes = np.unique(y_cat_train)
y_cat_test_classes_len = len(y_cat_test_classes)

In [15]:
categorical_y_train = to_categorical(y_cat_train, y_cat_train_classes_len)
categorical_y_test = to_categorical(y_cat_test, y_cat_test_classes_len)

model_name = "without_attention.h5"
mcp_save = ModelCheckpoint(filepath=model_name,
                           save_best_only=True,
                           monitor='val_loss',
                           mode='min',
                           verbose=1)

model = Sequential()
model.add(Embedding(vocab_size, 200, input_length=max_length))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True, name='lstm_layer')))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(y_cat_train_classes_len, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[categorical_accuracy])

model.summary()
batch_size = 8
epochs = 15
hist = model.fit(x_train_padded, categorical_y_train, batch_size=batch_size, epochs=epochs,
                           callbacks=mcp_save)

loss, acc = model.evaluate(x_test_padded, categorical_y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))
model.save("final_epoch_"+model_name)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 200)           69209000  
                                                                 
 bidirectional (Bidirectiona  (None, 25, 200)          241600    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 200)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 200)               40200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0

In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [19]:
from tensorflow.keras.models import load_model
model_name = "/content/final_epoch.h5"
model = load_model(model_name)
for layer in model.layers:
    print(layer.name)

embedding
bidirectional
global_max_pooling1d
dropout
dense
dropout_1
dense_1


In [18]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/final_epoch.h5" "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
